In [ ]:
### Non-Linear PDE solver in general domain
###     using Newton's Method
### L[u] = f
### Iterate u_{n+1} = u_n - J[u_n]^{-1} f ?
### Iterate u_{n+1} = u_n - J[u_n]^{-1} L[u_n] ?
###
### Second one doesn't work -- Our method cannot evaluate PDEs L[u_n]
### I don't think first one is true...

### Possible PDES

### viscous Burgers
###     u_t + u u_x = \mu u_xx
###
### Fisher-KPP
###     u_t = u_xx + u(1 - u)
###
### Allen-Cahn
###     u_t = u_xx + u - u^3


In [1]:
import time
import numpy as np
import math

import torch
import torch.nn as nn
import torch.optim as optim

import DRLPDE_functions.EvaluateWalkers
 
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import matplotlib.pyplot as plt

In [ ]:
### Frechet derivatives J[u] of 3 Linear Operators

### Probably no analytic solutions, going to have to simulate?
### Worried about finite element method, if the meshing is too hard

### Domain ellipse?

### Setup two neural networks
### u, h
### Two nested training: Train h with u, then replace u with u - h
###    This means I have to differentiate u using the diff operator L
###    This is kinda point-less...
###        Can't figure out Dynkin formula for this non-linearity


############## Save model and/or Load model ##############

# Physical Dimension
x_dim = 2
output_dim = 2

is_unsteady = True
input_dim = x_dim + is_unsteady

# Is there a true solution
exists_analytic_sol = False
# If there is a true solution, provide contour levels
plot_levels = np.linspace(-1,1,100)

def true_solution(X):
    pass

################# PDE Coefficients ########################

# PDE type:
pde_type = 'Nonlinear'

# Diffusion coefficient
mu = 1

# Forcing term
def forcing(X):
    f = torch.zeros( (X.size(0), output_dim), device=X.device)
    return f

################# Boundary and Initial Conditions ###########
# Use pytorch expressions to make boundary and initial conditions 
#
# To make different boundary conditions for each boundary
#     ensure the correct bdry_con is called when defining the boundaries

def bdry_con(X):
    u = torch.zeros( (X.size(0), output_dim), device=X.device)
    return u

def inlet_con(X):
    u = torch.zeros_like(X, device=X.device)
    
    u[:,0] = v0*torch.mul((L_height - X[:,1]),(L_height + X[:,1]))/(L_height**2)

    return u

#################  Make the domain  #######################
#     First define a bounding box containing your domain
#         Syntax: [ x interval, y interval, z interval ]
#         Points will be sampled through rejection sampling
#
#     Define each boundary
#         lines: [ 'line', point, normal, endpoints, bdry_condition ]
#         disk:  [ 'disk', centre, radius, endpoints, bdry_condition]
#         
#     Intersections of boundaries must be input manually
#         These should be ordered as points will be sampled from first to second
#         only 2 intersection points allowed
#         
#     Boundary condition is given by a function using pytorch expressions


boundingbox = [ [0, 5*L_height], [-L_height,L_height] ]

wall_left = {'type':'line',
             'point': [0, -L_height],
             'normal': [1,0],
             'endpoints': [ [0, -L_height], [0, L_height] ],
             'boundary_condition': inlet_con }

wall_top = { 'type':'line',
             'point': [0, L_height],
             'normal':  [0,-1],
             'endpoints': [ [0, L_height], [5*L_height, L_height] ],
             'boundary_condition': bdry_con }

wall_bot = {'type':'line',
             'point': [0,-L_height],
             'normal': [0, 1],
             'endpoints': [ [0, -L_height], [5*L_height, -L_height] ],
             'boundary_condition': bdry_con }

list_of_dirichlet_boundaries = [wall_left, wall_top, wall_bot ]
list_of_periodic_boundaries =[]